## DL model for Intel Classification
This is from one of the competition in Analytics Vidhya. Check the details [here](https://datahack.analyticsvidhya.com/contest/practice-problem-intel-scene-classification-challe/)

In [1]:
import numpy as np 
import pandas as pd 
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

import keras.backend as K
from keras.models import Sequential

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

%matplotlib inline

Using TensorFlow backend.


In [2]:
images = 'train/'
test = 'test_WyRytb0.csv'
labels = 'train.csv'
sample = 'sample_submission.csv'

In [3]:
# All the labels for the training data
y_train = pd.read_csv(labels)
y_train.head()

,image_name,label
0,0.jpg,0
1,1.jpg,4
2,2.jpg,5
3,4.jpg,0
4,7.jpg,4


In [4]:
# This will help in preparing the images for test data
y_test = pd.read_csv(test)
y_test.shape

(7301, 1)

## Preparing the Images for the model
We will process the image and make it according to our input for the model

In [5]:
def prepareImages(data, m):
    print("Preparing images")
    X_train = np.zeros((m, 150, 150, 3))
    count = 0
    
    for fig in data['image_name']:
        #load images into images of size 150*150(original size)
        #src = cv2.imread("train/"+fig,)
        #img = cv2.cvtColor(src,cv2.COLOR_BGR2RGB)
        img = image.load_img("train/"+fig, target_size=(150, 150, 3))
        x = image.img_to_array(img)
        x = preprocess_input(x)
        X_train[count] = x
        count += 1
        if(count%1000==0):
            print('images done :',count)
    
    return X_train

In [6]:
X_train = prepareImages(y_train,y_train.shape[0])
X_train /= 255

Preparing images
images done : 1000
images done : 2000
images done : 3000
images done : 4000
images done : 5000
images done : 6000
images done : 7000
images done : 8000
images done : 9000
images done : 10000
images done : 11000
images done : 12000
images done : 13000
images done : 14000
images done : 15000
images done : 16000
images done : 17000


In [7]:
X_train.shape

(17034, 150, 150, 3)

In [8]:
# fix random seed for reproducibility
random_seed = 7
np.random.seed(random_seed)

## Converting Output into one hot code

A one hot encoding is a representation of categorical variables as binary vectors. This first requires that the categorical values be mapped to integer values. Then, each integer value is represented as a binary vector that is all zero values except the index of the integer, which is marked with a 1 and as this is a multi classification problem so we can convert the output class values into one-hot format which is simply a binary matrix, i.e.

value 0 will be converted to one-hot format as [1, 0, 0, 0, 0, 0, 0, 0, 0]

value 1 will be converted to one-hot format as [0, 1, 0, 0, 0, 0, 0, 0, 0] etc

Here we have {'buildings' -> 0, 'forest' -> 1, 'glacier' -> 2, 'mountain' -> 3, 'sea' -> 4, 'street' -> 5 }

In [9]:
# one hot encode outputs'
y_train_label = np_utils.to_categorical(y_train['label'])
num_classes = y_train_label.shape[1]
num_classes

6

# Model using CNNs

In [10]:
def model():
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(150, 150, 3), activation='relu',data_format='channels_first'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(256, kernel_size = 4, activation='relu'))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dropout(0.25))
    model.add(Dense(6, activation='softmax'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

## Data Augmentation
We will use data augmentation for increasing our dataset and avoid overfitting.

In [11]:
# With data augmentation to prevent overfitting
datagen = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

## Train the model
- Define the model.
- We will split the dataset using sklearn train_test_split into training and validation set

In [12]:
model1 = model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 148, 1)        43232     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 148, 1)        4         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 146, 64)       640       
_________________________________________________________________
batch_normalization_2 (Batch (None, 30, 146, 64)       256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 73, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 73, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 71, 64)        36928     
__________

In [13]:
# DECREASE LEARNING RATE EACH EPOCH
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)
epochs = 70

x_train2, x_val, y_train2, y_val = train_test_split(X_train, y_train_label, test_size = 0.10, random_state=random_seed)
history = model1.fit_generator(datagen.flow(x_train2,y_train2, batch_size=32),
                              epochs = epochs, validation_data = (x_val,y_val),
                              verbose = 1, steps_per_epoch=(len(X_train)//32),validation_steps=(len(x_val)//32),callbacks=[annealer])

Epoch 1/70
532/532 [==============================] - 140s 263ms/step - loss: 0.5328 - acc: 0.8323 - val_loss: 0.3838 - val_acc: 0.8615
Epoch 2/70
532/532 [==============================] - 110s 206ms/step - loss: 0.3282 - acc: 0.8676 - val_loss: 0.3092 - val_acc: 0.8665
Epoch 3/70
532/532 [==============================] - 148s 278ms/step - loss: 0.2855 - acc: 0.8819 - val_loss: 0.2928 - val_acc: 0.8760
Epoch 4/70
532/532 [==============================] - 126s 237ms/step - loss: 0.2639 - acc: 0.8911 - val_loss: 0.2707 - val_acc: 0.8867
Epoch 5/70
532/532 [==============================] - 114s 215ms/step - loss: 0.2556 - acc: 0.8958 - val_loss: 0.2905 - val_acc: 0.8779
Epoch 6/70
532/532 [==============================] - 125s 234ms/step - loss: 0.2612 - acc: 0.8916 - val_loss: 0.2814 - val_acc: 0.8851
Epoch 7/70
532/532 [==============================] - 113s 212ms/step - loss: 0.2438 - acc: 0.8981 - val_loss: 0.3236 - val_acc: 0.8702
Epoch 8/70
532/532 [============================

Epoch 61/70
532/532 [==============================] - 141s 265ms/step - loss: 0.1223 - acc: 0.9503 - val_loss: 0.1804 - val_acc: 0.9270
Epoch 62/70
532/532 [==============================] - 134s 253ms/step - loss: 0.1218 - acc: 0.9509 - val_loss: 0.1788 - val_acc: 0.9281
Epoch 63/70
532/532 [==============================] - 123s 231ms/step - loss: 0.1251 - acc: 0.9492 - val_loss: 0.1786 - val_acc: 0.9276
Epoch 64/70
532/532 [==============================] - 122s 230ms/step - loss: 0.1226 - acc: 0.9503 - val_loss: 0.1768 - val_acc: 0.9302
Epoch 65/70
532/532 [==============================] - 128s 241ms/step - loss: 0.1232 - acc: 0.9506 - val_loss: 0.1779 - val_acc: 0.9292
Epoch 66/70
532/532 [==============================] - 122s 229ms/step - loss: 0.1207 - acc: 0.9521 - val_loss: 0.1819 - val_acc: 0.9269
Epoch 67/70
532/532 [==============================] - 119s 224ms/step - loss: 0.1203 - acc: 0.9509 - val_loss: 0.1781 - val_acc: 0.9277
Epoch 68/70
532/532 [====================

## To Do
Increase data using more data augmentation and also use binary__crossentropy in place of categorical for this multi label classification

In [14]:
model1.save_weights('model1_weights.h5')

## Predictions
Prepare the test Images and make the predictions

In [15]:
X_test = prepareImages(y_test,y_test.shape[0])
X_test /= 255

Preparing images
images done : 1000
images done : 2000
images done : 3000
images done : 4000
images done : 5000
images done : 6000
images done : 7000


In [18]:
result = np.zeros((X_test.shape[0],1))
result = model1.predict(X_test)
result = np.argmax(result,axis=1)
result = pd.Series(result,name='label')
submission = pd.concat([y_test,result],axis = 1)
submission.to_csv("first_try.csv",index=False)